In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from yellowbrick.regressor import ResidualsPlot
sns.set()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women) #verificando quantas mulheres sobreviveram nos dados de treino

In [ ]:
train_data.info()

In [ ]:
train_data["Pclass"].value_counts()

In [ ]:
train_data["Parch"].value_counts()

In [ ]:
test_data.info()

In [ ]:
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].mean())

In [ ]:
test_data.info()

In [ ]:
sns.boxplot(data=train_data, x="Survived", y="Age") #exibindo grafico de sobreviventes por idade

In [ ]:
(train_data
 .groupby("Survived")
 .agg({"Age": ["min", "max"]})) #descobrindo a idade mínima e máxima de quem sobreviveu e quem não

In [ ]:
sns.boxplot(data=train_data, x="Sex", y="Survived") #exibindo grafico de sobreviventes por sexo

**Verificando quantas pessoas sobreviveram nos dados de treino e quantas morreram**

In [ ]:
train_data["Survived"].value_counts(normalize=True) #

In [ ]:
train_data["Parch"].unique()

In [ ]:
train_data.duplicated(subset=["Survived","Pclass","Name","Sex", "Age", "SibSp", "Parch","Ticket","Fare","Cabin", "Embarked"]).sum() #verificando se há duplicadas

In [ ]:
train_data.describe()

In [ ]:
train_data["Cabin"].value_counts()

In [ ]:
train_data["Cabin"] = train_data["Cabin"].fillna("SEM CABINE")
test_data["Cabin"] = test_data["Cabin"].fillna("SEM CABINE")
test_data["Parch"] = test_data["Parch"].fillna(0)

In [ ]:
train_data.head()

In [ ]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()

train_data.loc[:,"Age_imp"] = my_imputer.fit_transform(train_data.loc[:,["Age"]])
test_data.loc[:,"Age_imp"] = my_imputer.fit_transform(test_data.loc[:,["Age"]])
#train_data["Age"]=

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(data=train_data[["Survived","Fare"]], y="Fare", x="Survived")
plt.xticks(rotation =45)

In [ ]:
sns.set_style("white")
plt.figure(figsize=(12,6))
sns.violinplot(data=train_data, x = "Survived", y= "SibSp")

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(data=train_data[["Survived","Parch"]], y="Parch", x="Survived")
plt.xticks(rotation =45)

In [ ]:
def removendo_outliers (df,feat):
    q1, q3 = df[feat].quantile([.25,.75])
    iqr = q3 - q1
    lower_bound = max(q1-1.5*iqr,0)
    upper_bound = (q3+1.5*iqr)
    len_before = (~df[feat].isnull()).sum()
    df[~df[feat].between(lower_bound,upper_bound)] = np.nan
    len_after = (~df[feat].isnull()).sum()
    print (f"Removidos {(1 -(len_after/len_before))*100}% dos dados")
    return df

In [ ]:
sns.heatmap(train_data.corr(), annot=True, vmin=-1, vmax=1) 

In [ ]:
test_data.info()

In [ ]:
train_data["Fare"].isnull().sum()

In [ ]:
y = train_data["Survived"]

features = ["Pclass", "SibSp", "Parch","Age_imp","Fare","Sex"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

In [ ]:
train_data["Age_imp"]=removendo_outliers(train_data,"Age_imp")
train_data["SibSp"]=removendo_outliers(train_data,"SibSp")

In [ ]:
X.isnull().sum()

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(data=train_data[["Survived","Age_imp"]], y="Age_imp", x="Survived")
plt.xticks(rotation =45) #verificando se funcionou

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.3, stratify=y, shuffle=True)

In [ ]:
from scikitplot.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from scikitplot.metrics import plot_cumulative_gain, plot_lift_curve, plot_ks_statistic
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from scikitplot.metrics import plot_confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

clf = DecisionTreeClassifier(max_depth=8, min_samples_leaf=20)
clf = clf.fit(X,y)
y_pred_clf = clf.predict(X_valid)
print(classification_report(y_valid, y_pred_clf))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model=KNeighborsClassifier(n_neighbors=3)

model.fit(X,y)
y_pred_kneigh= model.predict(X_valid)
#print(X_test,y_pred_kneigh)
print(classification_report(y_valid, y_pred_kneigh))

In [ ]:
model2 = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model2.fit(X, y)
y_pred_rndForest = model.predict(X_valid)
print(classification_report(y_valid, y_pred_rndForest))

In [ ]:
y_pred=[y_pred_clf, y_pred_kneigh, y_pred_rndForest]

In [ ]:
plot_confusion_matrix(y_valid, y_pred[0])

In [ ]:
plot_confusion_matrix(y_valid, y_pred[1])

In [ ]:
X_test.info()

In [ ]:
X_test = X_test.fillna(X_test.mean())

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', DecisionTreeClassifier(max_depth=8, min_samples_leaf=20))
                             ])


my_pipeline2 = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', KNeighborsClassifier(n_neighbors=3))
                             ])

my_pipeline3 = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

In [ ]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = cross_val_score(my_pipeline, X_train, y_train,
                              cv=5,
                              scoring='roc_auc')

scores2 = cross_val_score(my_pipeline2, X_train, y_train,
                              cv=5,
                              scoring='roc_auc')

scores3 = cross_val_score(my_pipeline3, X_train, y_train,
                              cv=5,
                              scoring='roc_auc')

Avr_score1=scores.mean()
Avr_score2=scores2.mean()
Avr_score3=scores3.mean()


print("AUC DecisionTreeClassifier:\n", scores.mean(),scores.std())
print("AUC KNeighborsClassifier:\n", scores2.mean(), scores2.std())
print("AUC RandomForestRegressor:\n", scores3.mean(), scores3.std())

**Random Forest tem melhor média absoluta de erro, então esse será o modelo escolhido.**

In [ ]:
X_test.info()

In [ ]:
plot_confusion_matrix(y_valid, y_pred[2])

In [ ]:
my_pipeline.fit(X, y)
my_pipeline2.fit(X, y)
my_pipeline3.fit(X, y) #fazer .fit do modelo que tiver melhor performance

In [ ]:
y_pred_clf = my_pipeline.predict(X_test)
y_pred_kneigh= my_pipeline2.predict(X_test)
y_pred_rndForest = my_pipeline3.predict(X_test)

In [ ]:
#plot_confusion_matrix(y_train2, y_pred)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_pred_kneigh})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")